<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural - CEIA
## Alumno: Gonzalo Gontad
## Cohorte: 8va

### En el presente ejercicio se desarrolló un bot de preguntas y respuestas utilizando SPACY para el procesamiento del corpus, DNN y TF-IDF para la búsqueda de respuestas.


### 1 - Instalar dependencias
Para poder utilizar Spacy en castellano es necesario agregar la librería "spacy-stanza" para lematizar palabras en español.

In [1]:
import json
import string
import random 
import numpy as np

import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout

2023-05-17 19:45:06.177392: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-17 19:45:06.223602: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-17 19:45:06.224455: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 19:45:06.886476: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Si se quiere entrenar el modelo desde cero, se debe cambiar el valor a True
# Si se quiere cargar un modelo ya entrenado, se debe cambiar el valor a False
train_model = False

In [3]:
import stanza
import spacy_stanza

# Vamos a usar SpaCy-Stanza. Stanza es una librería de NLP de Stanford
# SpaCy armó un wrapper para los pipelines y modelos de Stanza
# https://stanfordnlp.github.io/stanza/

# Descargar el diccionario en español y armar el pipeline de NLP con spacy
stanza.download("es")
nlp = spacy_stanza.load_pipeline("es")

/home/gonzalo/anaconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-17 19:45:09 INFO: Downloading default packages for language: es (Spanish)...
2023-05-17 19:45:10 INFO: File exists: /home/gonzalo/stanza_resources/es/default.zip.
2023-05-17 19:45:13 INFO: Finished downloading models and saved to /home/gonzalo/stanza_resources.
2023-05-17 19:45:13 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2023-05-17 19:45:13 INFO: Use device: cpu
2023-05-17 19:45:13 INFO: Loading: tokenize
2023-05-17 19:45:13 INFO: Loading: mwt
2023-05-17 19:45:13 INFO: Loading: pos
2023-05-17 19:45:13 INFO: Loadin

### 2 - Herramientas de preprocesamiento de datos
Entre las tareas de procesamiento de texto en español se implementa:
- Quitar acentos y caracteres especiales
- Quitar números
- Quitar símbolos de puntuación

In [4]:
import re
import unicodedata

# El preprocesamento en castellano requiere más trabajo

# Referencia de regex:
# https://docs.python.org/3/library/re.html

def preprocess_clean_text(text):    
    # sacar tildes de las palabras:
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # quitar caracteres especiales
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]' # [^ : ningún caracter de todos estos
    # (termina eliminando cualquier caracter distinto de los del regex)
    text = re.sub(pattern, '', text)
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' # igual al anterior pero sin cifras numéricas
    # quitar números
    text = re.sub(pattern, '', text)
    # quitar caracteres de puntuación
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

In [5]:
text = "personas Ideas! estás cosas y los peces y los murciélagos"

# Antes de preprocesar los datos se pasa a minúsculas todo el texto
preprocess_clean_text(text.lower())

'personas ideas estas cosas y los peces y los murcielagos'

In [6]:
# Ejemplo de como fuciona
text = "hola personas Ideas! estás cosas y los peces y los murciélagos"

# Antes de preprocesar los datos se pasa a minúsculas todo el texto
tokens = nlp(preprocess_clean_text(text.lower()))
print("tokens:", tokens)
print("Lematización de cada token:")
for token in tokens:
    print([token, token.lemma_])

tokens: hola personas ideas estas cosas y los peces y los murcielagos
Lematización de cada token:
[hola, 'holar']
[personas, 'persona']
[ideas, 'idea']
[estas, 'este']
[cosas, 'cosa']
[y, 'y']
[los, 'el']
[peces, 'pez']
[y, 'y']
[los, 'el']
[murcielagos, 'murcielago']


### 3 - Diccionario de entrada

El siguiente dataset fue generado utilizando ChatGPT unicamente con el fin de tener un dataset de entrada para el bot. Este puede presentar errores por lo que las respuestas no deben ser tomadas como verdaderas.

In [7]:
# Dataset en formato JSON que representa las posibles preguntas (patterns)
# y las posibles respuestas por categoría (tag)
# Los "patterns" van a formar el corpus para entrenar el clasificador que clasifica en tags
# "respones" son las respuestas predeterminadas posibles para cada tag
dataset = {"intents": [
            {
              "tag": "bienvenida",
              "patterns": ["Hola", "¿Cómo estás?", "¿Qué tal?"],
              "responses": ["Hola!", "Hola, ¿Cómo estás?", "¡Bienvenido!", "¡Saludos!"]
            },
            {"tag": "nombre",
            "patterns": ["¿Cúal es tu nombre?", "¿Quién sos?"],
            "responses": ["Soy un chatbot que responde preguntas sobre el libro La Guía del Viajero Intergaláctico", "Puedes llamarme Marvin"]
            },
            {
              "tag": "el_libro",
              "patterns": [ "¿De qué trata el libro?", "¿Me puedes contar sobre el libro?","¿De qué trata la historia?", "Resumen del libro."],
              "responses": ["La Guía del Viajero Intergaláctico es una novela de ciencia ficción escrita por Douglas Adams. Cuenta la historia de un humano llamado Arthur Dent y su amigo alienígena, Ford Prefect, quienes se embarcan en una aventura intergaláctica después de que la Tierra es destruida para dar paso a una autopista hiperespacial. La Guía del Viajero Intergaláctico es una herramienta útil para los viajeros galácticos, ya que contiene información sobre los planetas, especies y culturas del universo."],
            },
            {
              "tag": "la_guia",
              "patterns": ["¿Qué es la Guía del Viajero Intergaláctico?", "¿Me puedes contar sobre la Guía del Viajero Intergaláctico?"],
              "responses": ["La Guía del Viajero Intergaláctico es una herramienta útil para los viajeros galácticos, ya que contiene información sobre los planetas, especies y culturas del universo.", "Es una enciclopedia de viajes intergallacticos."],
            },
            {
              "tag": "personajes",
              "patterns": ["¿Quiénes son los personajes principales de La Guía del Viajero Intergaláctico?", "¿Qué personajes aparecen en el libro?", "¿Me puedes hablar sobre los personajes de la historia?", "Nombres de personajes."],
              "responses": ["Los personajes principales son Arthur Dent, un humano que es arrastrado a una aventura intergaláctica; Ford Prefect, un extraterrestre que trabaja para la Guía del Viajero Intergaláctico; Zaphod Beeblebrox, el excéntrico presidente de la galaxia; Marvin, un robot paranoide y deprimido; y Trillian, la única otra persona de la Tierra que sobrevive a su destrucción.","Arthur, Ford, Zaphod, Marvin y Trillian."],
            },
            {
              "tag": "humor",
              "patterns": ["¿El libro es divertido?", "¿Tiene La Guía del Viajero Intergaláctico un toque de humor?", "¿Es el libro una comedia?"],
              "responses": ["Sí, La Guía del Viajero Intergaláctico es una novela de ciencia ficción con un toque de humor absurdo. Douglas Adams tenía un gran sentido del humor y eso se refleja en la historia y en los personajes."],
            },
            {
              "tag": "adaptaciones",
              "patterns": ["¿Hay alguna película o serie basada en La Guía del Viajero Intergaláctico?", "¿Existe alguna adaptación del libro?", "¿Me recomiendas alguna adaptación del libro?"],
              "responses": ["Sí, hay una película de 2005 y una serie de televisión de 1981 basadas en La Guía del Viajero Intergaláctico. También hay adaptaciones de radio y teatro. Cada una de ellas tiene su propio estilo y enfoque, pero todas son divertidas y entretenidas."],
            },
            {
              "tag": "autor",
              "patterns": ["¿Quién escribió el libro?", "Autor del libro."],
              "responses": ["Douglas Adams.", "Adams fue el autor."],
            },
            {
              "tag": "agradecer",
              "patterns": ["Gracias", "Muchas gracias", "Gracias por la ayuda"],
              "responses": ["¡De nada!", "¡No hay problema!", "¡Un placer ayudar!"],
            },
            {
              "tag": "frase",
              "patterns": ["¿Cuál es la frase más famosa del libro?", "¿Me puedes decir una frase del libro?", "¿Cuál es la frase más famosa de La Guía del Viajero Intergaláctico?"],
              "responses": ["No entiendo nada, pero soy capaz de explicártelo.", "La respuesta a la vida, el universo y todo lo demás es 42."],
            },
            {    
              "tag": "despedida",    
              "patterns": ["Adiós", "Hasta luego", "Nos vemos", "Chau", "Hasta pronto"],
              "responses": ["Hasta luego, y gracias por el pescado"],
            }                    

]}

### 4 - Preprocesamiento y armado del dataset

In [8]:
# Datos que necesitaremos, las palabras o vocabulario
words = []
classes = []
doc_X = []
doc_y = []

# Por cada intención (intents) debemos tomar los patrones que la caracterízan
# a esa intención y transformarla a tokens para almacenar en doc_X

# El tag de cada intención se almacena como doc_Y (la clase a predecir)
# En `words` vamos a guardar el vocabulario
# En `class` las posibles clases o tags

for intent in dataset["intents"]:
    for pattern in intent["patterns"]:
        # trasformar el patron a tokens
        tokens = nlp(preprocess_clean_text(pattern.lower()))
        # lematizar los tokens
        for token in tokens:            
            words.append(token.lemma_)
        
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # Agregar el tag a las clases
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# Elminar duplicados con "set" y ordenar el vocubulario y las clases por orden alfabético
words = sorted(set(words))
classes = sorted(set(classes))

/tmp/ipykernel_9033/2762758537.py:17: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  tokens = nlp(preprocess_clean_text(pattern.lower()))
/tmp/ipykernel_9033/2762758537.py:17: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['resumen', 'de', 'el', 'libro']
Entities: []
  tokens = nlp(preprocess_clean_text(pattern.lower()))
/tmp/ipykernel_9033/2762758537.py:17: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['que', 'es', 'la', 'guia', 'de', 'el', 'viajero', 'intergalactico']
Entities: []
  tokens = nlp(preprocess_clean_text(pattern.lower()))
/tmp/ipykernel_9033/2762758537.py:17: UserWarning: Can't set named entities because of multi-word to

In [9]:
print("words:", words)
print("classes:", classes)
print("doc_X:", doc_X)
print("doc_y:", doc_y)

words: ['adaptacion', 'adios', 'alguno', 'aparecer', 'autor', 'ayuda', 'basado', 'chau', 'comedia', 'como', 'contar', 'cual', 'de', 'decir', 'divertido', 'el', 'en', 'escribir', 'este', 'existir', 'famoso', 'frase', 'gracias', 'guia', 'haber', 'hablar', 'hasta', 'historia', 'holar', 'humor', 'intergalactico', 'libro', 'luego', 'mas', 'mucho', 'nombre', 'o', 'pelicula', 'personaje', 'poder', 'por', 'principal', 'pronto', 'que', 'quien', 'recomendar', 'resumen', 'ser', 'serie', 'sobre', 'tal', 'tener', 'toque', 'tratar', 'tu', 'uno', 'ver', 'viajero', 'yo']
classes: ['adaptaciones', 'agradecer', 'autor', 'bienvenida', 'despedida', 'el_libro', 'frase', 'humor', 'la_guia', 'nombre', 'personajes']
doc_X: ['Hola', '¿Cómo estás?', '¿Qué tal?', '¿Cúal es tu nombre?', '¿Quién sos?', '¿De qué trata el libro?', '¿Me puedes contar sobre el libro?', '¿De qué trata la historia?', 'Resumen del libro.', '¿Qué es la Guía del Viajero Intergaláctico?', '¿Me puedes contar sobre la Guía del Viajero Interga

In [10]:
# Tamaño del vocabulario
print("Vocabulario:", len(words))

Vocabulario: 59


In [11]:
# Cantidad de tags
print("Tags:", len(classes))

Tags: 11


In [12]:
# Transformar doc_X en bag of words por oneHotEncoding
# Transformar doc_Y en un vector de clases multicategórico con oneHotEncoding

training = []
out_empty = [0] * len(classes)

for idx, doc in enumerate(doc_X):
    # Transformar la pregunta (input) en tokens y lematizar
    text = []
    tokens = nlp(preprocess_clean_text(doc.lower()))
    for token in tokens:
        text.append(token.lemma_)

    # Transformar los tokens en "Bag of words" (arrays de 1 y 0)
    bow = []
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    
    # Crear el array de salida (class output) correspondiente
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1

    print("X:", bow, "y:", output_row)
    training.append([bow, output_row])

# Mezclar los datos
random.shuffle(training)
training = np.array(training, dtype=object)
# Dividir en datos de entrada y salida
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] y: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] y: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0] y: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0] y: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

/tmp/ipykernel_9033/220604768.py:10: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  tokens = nlp(preprocess_clean_text(doc.lower()))
/tmp/ipykernel_9033/220604768.py:10: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['resumen', 'de', 'el', 'libro']
Entities: []
  tokens = nlp(preprocess_clean_text(doc.lower()))
/tmp/ipykernel_9033/220604768.py:10: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['que', 'es', 'la', 'guia', 'de', 'el', 'viajero', 'intergalactico']
Entities: []
  tokens = nlp(preprocess_clean_text(doc.lower()))


X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] y: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]


/tmp/ipykernel_9033/220604768.py:10: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['me', 'puedes', 'contar', 'sobre', 'la', 'guia', 'de', 'el', 'viajero', 'intergalactico']
Entities: []
  tokens = nlp(preprocess_clean_text(doc.lower()))


X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1] y: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]


...

X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] y: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


/tmp/ipykernel_9033/220604768.py:10: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['me', 'puedes', 'decir', 'una', 'frase', 'de', 'el', 'libro']
Entities: []
  tokens = nlp(preprocess_clean_text(doc.lower()))


X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1] y: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


/tmp/ipykernel_9033/220604768.py:10: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['cual', 'es', 'la', 'frase', 'mas', 'famosa', 'de', 'la', 'guia', 'de', 'el', 'viajero', 'intergalactico']
Entities: []
  tokens = nlp(preprocess_clean_text(doc.lower()))


X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] y: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
X: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] y: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] y: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1] y: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
X: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

### 5 - Entrenamiento del modelo

In [13]:
# Shape de entrada y salida
input_shape = (train_X.shape[1],)
output_shape = train_y.shape[1]
print("input:", input_shape, "output:", output_shape)

input: (59,) output: 11


In [14]:
# Entrenamiento del modelo DNN
# - Modelo secuencial
# - Con regularización
# - softmax y optimizador Adam
if train_model:
    model = Sequential()
    model.add(Dense(128, input_shape=input_shape, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(output_shape, activation = "softmax"))

    model.compile(loss='categorical_crossentropy',
                optimizer="Adam",
                metrics=["accuracy"])
    print(model.summary())

In [15]:
if train_model:
    hist = model.fit(x=train_X, y=train_y, epochs=2000)

In [16]:
import matplotlib.pyplot as plt
import seaborn as sns

if train_model:
    # Entrenamiento
    epoch_count = range(1, len(hist.history['accuracy']) + 1)
    sns.lineplot(x=epoch_count,  y=hist.history['accuracy'], label='train')
    plt.show()

In [17]:
# Guardar lo necesario para poder re-utilizar este modelo en el futuro
# el vocabulario utilizado (words)
# las posibles clases
# el modelo
import pickle

if train_model:
    pickle.dump(words, open('words.pkl','wb'))
    pickle.dump(classes, open('classes.pkl','wb'))
    model.save('chatbot_model.h5')

# TF-IDF + Similaridad coseno

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
class tf_idf_predictor ():
    def __init__(self, corpus):
        self.corpus = corpus
        self.vectorizer = TfidfVectorizer()
        self.X = self.vectorizer.fit_transform(self.corpus)

    def predict(self, text):
        query = self.vectorizer.transform([text])
        cs = cosine_similarity(query, self.X)
        #print(cs)
        return self.corpus[np.argmax(cs)], np.argmax(cs)

In [20]:
def get_response(intents_list, intents_json):
    tag = intents_list[0] # tomar el tag con el mejor valor softmax
    list_of_intents = intents_json["intents"] # intents_json es todo el dataset
    for i in list_of_intents: 
        if i["tag"] == tag: # buscar el tag correspoindiente y dar una respuesta predeterminada aleatoria 
            result = random.choice(i["responses"])
            break
    return result

In [21]:
corpus_tf_idf = []
corpus_tags_tf_idf = []
for intent in dataset['intents']:
    corpus_tf_idf.extend(intent['patterns'])
    corpus_tags_tf_idf.extend([intent['tag']] * len(intent['patterns']))
tf_idf_pred = tf_idf_predictor(corpus_tf_idf)

In [22]:
#obtengo una respuesta
question = "Autor"
most_sim, corpus_tag_idx = tf_idf_pred.predict(question)
#busco el tag de la respuesta
get_response([corpus_tags_tf_idf[corpus_tag_idx]], dataset)


'Adams fue el autor.'

### 6 - Testing y validación

In [23]:
# convertir texto de entrada del usuario a tokens
def text_to_tokens(text):
    lemma_tokens = []
    tokens = nlp(preprocess_clean_text(text.lower()))
    for token in tokens:
        lemma_tokens.append(token.lemma_)
    #print(lemma_tokens)
    return lemma_tokens

# transformar el texto de entrada tokenizado a una representación OHE
def bag_of_words(text, vocab): 
    tokens = text_to_tokens(text)
    bow = [0] * len(vocab)
    for w in tokens: 
        for idx, word in enumerate(vocab):
            if word == w: 
                bow[idx] = 1
    #print(bow)
    return np.array(bow)

# usar modelo con la entrada en OHE y los labels posibles (tags)
def pred_class(text, vocab, labels): 
    bow = bag_of_words(text, vocab)
    words_recognized = sum(bow)

    return_list = []
    if words_recognized > 0: # sólo si reconoció alguna palabra del vocabulario
        result = model.predict(np.array([bow]))[0] # es un array de softmax
        thresh = 0.2
        # filtrar aquellas entradas menores al umbral `thresh`
        y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
        # ordenar keys de acuerdo al valor softmax
        y_pred.sort(key=lambda x: x[1], reverse=True)
    
        # return_list es una lista de los labels de mayor a menor
        for r in y_pred:
            return_list.append(labels[r[0]])
            #print(labels[r[0]], r[1])

    # si no reconoció palabras del vocabulario se devuelve una lista vacía
    return return_list

# obtener una respuesta predeterminada 
def get_response(intents_list, intents_json):
    tag = intents_list[0] # tomar el tag con el mejor valor softmax
    list_of_intents = intents_json["intents"] # intents_json es todo el dataset
    for i in list_of_intents: 
        if i["tag"] == tag: # buscar el tag correspoindiente y dar una respuesta predeterminada aleatoria 
            result = random.choice(i["responses"])
            break
    return result

In [24]:
if train_model==False:
    #loas tensorflow model
    model = tf.keras.models.load_model('chatbot_model.h5')
    #load classes pickle
    classes = pickle.load(open('classes.pkl','rb'))
    #load words pickle
    words = pickle.load(open('words.pkl','rb'))


## Pruebo ambos metodos

In [44]:
import gradio as gr
import pandas as pd
questions_responses = pd.DataFrame(columns=['question', 'DNN', 'TF-IDF'])

def bot_response(human_text):
    intents = pred_class(human_text, words, classes)
    if len(intents) > 0:
        resp_a = get_response(intents, dataset)
    else: # si no hubo ningún resultado que supere el umbral
        resp_a = "Perdón, no comprendo la pregunta."
    most_sim, corpus_tag_idx = tf_idf_pred.predict(human_text)
    resp_b = get_response([corpus_tags_tf_idf[corpus_tag_idx]], dataset)
    print("Q:", human_text)  
    print("A:", resp_a)
    print("B:", resp_b)
    questions_responses.loc[len(questions_responses)] = [human_text, resp_a, resp_b]
    return resp_a, resp_b

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs=["text","text"],
    layout="vertical",
    title="Chatbot",
    )


iface.launch(debug=True)

/home/gonzalo/anaconda3/envs/nlp/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: `layout` parameter is deprecated, and it has no effect
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 22ms/step
Q: Hola
A: ¡Bienvenido!
B: Hola!
1/1 [==============================] - 0s 33ms/step
Q: ¿Cual es tu nombre?
A: Soy un chatbot que responde preguntas sobre el libro La Guía del Viajero Intergaláctico
B: Puedes llamarme Marvin
1/1 [==============================] - 0s 20ms/step


/tmp/ipykernel_6776/3251553722.py:4: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  tokens = nlp(preprocess_clean_text(text.lower()))
/tmp/ipykernel_6776/3251553722.py:4: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['quien', 'es', 'el', 'autor', 'de', 'el', 'libro']
Entities: []
  tokens = nlp(preprocess_clean_text(text.lower()))


Q: ¿Quien es el autor del libro?
A: Adams fue el autor.
B: Douglas Adams.
1/1 [==============================] - 0s 16ms/step
Q: ¿Que es la guia para el viajero intergalactico?
A: La Guía del Viajero Intergaláctico es una herramienta útil para los viajeros galácticos, ya que contiene información sobre los planetas, especies y culturas del universo.
B: La Guía del Viajero Intergaláctico es una herramienta útil para los viajeros galácticos, ya que contiene información sobre los planetas, especies y culturas del universo.
1/1 [==============================] - 0s 16ms/step
Q: ¿De que trata el libro?
A: La Guía del Viajero Intergaláctico es una novela de ciencia ficción escrita por Douglas Adams. Cuenta la historia de un humano llamado Arthur Dent y su amigo alienígena, Ford Prefect, quienes se embarcan en una aventura intergaláctica después de que la Tierra es destruida para dar paso a una autopista hiperespacial. La Guía del Viajero Intergaláctico es una herramienta útil para los viajero

...

Q: ¿Puedes decirme alguna frase del libro?
A: La respuesta a la vida, el universo y todo lo demás es 42.
B: No entiendo nada, pero soy capaz de explicártelo.
1/1 [==============================] - 0s 15ms/step


/tmp/ipykernel_6776/3251553722.py:4: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  tokens = nlp(preprocess_clean_text(text.lower()))
/tmp/ipykernel_6776/3251553722.py:4: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['quien', 'es', 'el', 'escritor', 'de', 'el', 'libro']
Entities: []
  tokens = nlp(preprocess_clean_text(text.lower()))


Q: ¿quien es el escritor del libro?
A: Soy un chatbot que responde preguntas sobre el libro La Guía del Viajero Intergaláctico
B: Sí, La Guía del Viajero Intergaláctico es una novela de ciencia ficción con un toque de humor absurdo. Douglas Adams tenía un gran sentido del humor y eso se refleja en la historia y en los personajes.
1/1 [==============================] - 0s 18ms/step
Q: ¿Como te llamas?
A: Hola, ¿Cómo estás?
B: ¡Saludos!
1/1 [==============================] - 0s 21ms/step


/tmp/ipykernel_6776/3251553722.py:4: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  tokens = nlp(preprocess_clean_text(text.lower()))
/tmp/ipykernel_6776/3251553722.py:4: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['cual', 'es', 'la', 'tematica', 'de', 'el', 'libro']
Entities: []
  tokens = nlp(preprocess_clean_text(text.lower()))


Q: ¿Cual es la tematica del libro?
A: No entiendo nada, pero soy capaz de explicártelo.
B: No entiendo nada, pero soy capaz de explicártelo.
1/1 [==============================] - 0s 18ms/step
Q: ¿Para que sirve la guia?
A: Es una enciclopedia de viajes intergallacticos.
B: No entiendo nada, pero soy capaz de explicártelo.
1/1 [==============================] - 0s 19ms/step
Q: Adios
A: Hasta luego, y gracias por el pescado
B: Hola, ¿Cómo estás?
1/1 [==============================] - 0s 20ms/step
Q: Chau
A: Hasta luego, y gracias por el pescado
B: Hasta luego, y gracias por el pescado
1/1 [==============================] - 0s 20ms/step
Q: Gracias
A: ¡Un placer ayudar!
B: ¡Un placer ayudar!
Keyboard interruption in main thread... closing server.


### Resumen de preguntas y respuestas

In [45]:
questions_responses

,question,DNN,TF-IDF
0,Hola,¡Bienvenido!,Hola!
1,¿Cual es tu nombre?,Soy un chatbot que responde preguntas sobre el...,Puedes llamarme Marvin
2,¿Quien es el autor del libro?,Adams fue el autor.,Douglas Adams.
3,¿Que es la guia para el viajero intergalactico?,La Guía del Viajero Intergaláctico es una herr...,La Guía del Viajero Intergaláctico es una herr...
4,¿De que trata el libro?,La Guía del Viajero Intergaláctico es una nove...,La Guía del Viajero Intergaláctico es una nove...
5,¿Cuales son los personajes principales?,"Los personajes principales son Arthur Dent, un...","Los personajes principales son Arthur Dent, un..."
6,¿Puedes decirme alguna frase del libro?,"La respuesta a la vida, el universo y todo lo ...","No entiendo nada, pero soy capaz de explicártelo."
7,¿quien es el escritor del libro?,Soy un chatbot que responde preguntas sobre el...,"Sí, La Guía del Viajero Intergaláctico es una ..."
8,¿Como te llamas?,"Hola, ¿Cómo estás?",¡Saludos!
9,¿Cual es la tematica del libro?,"No entiendo nada, pero soy capaz de explicártelo.","No entiendo nada, pero soy capaz de explicártelo."


### 7 - Conclusiones
En el resumen se puede ver que tanto el predictor mediante redes neuronales como el predictor mediante TF-IDF + Similaridad coseno tienen un desempeño similar.
Los principales errores se deben al uso de sinonimos o palabras similares que no estan en el diccionario de entrada, como en el caso de las preguntas 7, 8, 9 y 10.
Tambien se pueden ver casos mas graves como el saludo en el punto 11 que  esta en el diccionario de entrada pero el prdictor TF-IDF no obtuvo la respuesta correcta.

### 8 - Mejoras
Para mejorar el desempeño se podria utilizar un dataset mas grande y con mas variedad de preguntas para cada respuesta y mayor cantidad de sinonimos.
